In [ ]:
Exploratory Data Analysis

In [98]:
!pip install --upgrade google-cloud-bigquery pandas-gbq pyarrow

In [99]:
import os
import pandas as pd
import sqlite3
from google.cloud import bigquery
# Create BigQuery client
client = bigquery.Client(project="main-cocoa-308521")

'/Users/oscartran/Peek'

For each bike ID, compute the longest streak of consecutive days it was used.

In [101]:
# Define your query — example: Citbike Trip dataset
query = """
WITH bike_days AS (
  SELECT
    bikeid,
    DATE(starttime) AS ride_date
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE bikeid IS NOT NULL
  GROUP BY bikeid, ride_date
),

streaks AS (
  SELECT
    bikeid,
    ride_date,
    DATE_SUB(ride_date, INTERVAL ROW_NUMBER() OVER (PARTITION BY bikeid ORDER BY ride_date) DAY) AS streak_group
  FROM bike_days
),

streak_lengths AS (
  SELECT
    bikeid,
    streak_group,
    COUNT(*) AS streak_length
  FROM streaks
  GROUP BY bikeid, streak_group
)

SELECT
  bikeid,
  MAX(streak_length) AS longest_streak
FROM streak_lengths
GROUP BY bikeid
ORDER BY longest_streak DESC
"""

# Run the query
streak_query_job = client.query(query)

# Convert to pandas DataFrame
results = streak_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


       bikeid  longest_streak
0       17333             131
1       26324             116
2       18897             114
3       15870             105
4       23273             105
...       ...             ...
17677   21758               1
17678   30402               1
17679   31859               1
17680   29833               1
17681   29598               1

[17682 rows x 2 columns]


Which stations show the biggest difference between weekday and weekend usage?

In [195]:
# Define your query — example: Citbike Trip dataset
query = """
WITH day_type_counts AS (
  SELECT
    start_station_name,
    CASE
      WHEN EXTRACT(DAYOFWEEK FROM starttime) IN (1, 7) THEN 'weekend'
      ELSE 'weekday'
    END AS day_type,
    COUNT(*) AS trip_count
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE start_station_name IS NOT NULL AND end_station_name <> ''
  GROUP BY start_station_name, day_type
),

pivoted AS (
  SELECT
    start_station_name,
    SUM(CASE WHEN day_type = 'weekday' THEN trip_count ELSE 0 END) AS weekday_trips,
    SUM(CASE WHEN day_type = 'weekend' THEN trip_count ELSE 0 END) AS weekend_trips
  FROM day_type_counts
  GROUP BY start_station_name
)

SELECT
  start_station_name,
  weekday_trips,
  weekend_trips,
  ABS(weekday_trips - weekend_trips) AS usage_diff
FROM pivoted
ORDER BY usage_diff DESC
LIMIT 20
"""

# Run the query
day_query_job = client.query(query)

# Convert to pandas DataFrame
results = day_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


         start_station_name  weekday_trips  weekend_trips  usage_diff
0     Pershing Square North         393615          44462      349153
1           8 Ave & W 31 St         348074          53480      294594
2           W 41 St & 8 Ave         270784          40619      230165
3        Broadway & E 22 St         295823          71371      224452
4           8 Ave & W 33 St         276441          53937      222504
5           W 21 St & 6 Ave         308169          95626      212543
6        E 17 St & Broadway         317713         105621      212092
7           W 38 St & 8 Ave         242224          31357      210867
8           W 31 St & 7 Ave         236030          34265      201765
9     Lafayette St & E 8 St         285704          86551      199153
10       E 47 St & Park Ave         210173          15810      194363
11          W 33 St & 7 Ave         212412          36929      175483
12       Broadway & E 14 St         254501          90045      164456
13       E 32 St & P

How has the average trip duration changed month-over-month for the 20 busiest stations?

In [103]:
# Define your query — example: Citbike Trip dataset
query = """
WITH trip_counts AS (
  SELECT
    start_station_name,
    COUNT(*) AS trip_count
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  WHERE start_station_name IS NOT NULL
  GROUP BY start_station_name
  ORDER BY trip_count DESC
  LIMIT 20
),

monthly_avg_duration AS (
  SELECT
    t.start_station_name,
    EXTRACT(YEAR FROM t.starttime) AS trip_year,
    EXTRACT(MONTH FROM t.starttime) AS trip_month,
    AVG(TIMESTAMP_DIFF(t.stoptime, t.starttime, MINUTE)) AS avg_duration_min
  FROM `bigquery-public-data.new_york_citibike.citibike_trips` t
  JOIN trip_counts b
    ON t.start_station_name = b.start_station_name
  WHERE t.start_station_name IS NOT NULL
    AND t.stoptime IS NOT NULL
    AND t.starttime IS NOT NULL
  GROUP BY start_station_name, trip_year, trip_month
)

SELECT
  start_station_name,
  trip_year,
  trip_month,
  avg_duration_min
FROM monthly_avg_duration
ORDER BY start_station_name, trip_year, trip_month

"""

# Run the query
trip_duration_query_job = client.query(query)

# Convert to pandas DataFrame
results = trip_duration_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


        start_station_name  trip_year  trip_month  avg_duration_min
0          8 Ave & W 31 St       2013           7         14.774972
1          8 Ave & W 31 St       2013           8         14.472288
2          8 Ave & W 31 St       2013           9         14.244704
3          8 Ave & W 31 St       2013          10         13.101643
4          8 Ave & W 31 St       2013          11         12.611035
..                     ...        ...         ...               ...
984  West St & Chambers St       2018           1         14.391111
985  West St & Chambers St       2018           2         15.623875
986  West St & Chambers St       2018           3         16.093209
987  West St & Chambers St       2018           4         20.205197
988  West St & Chambers St       2018           5         20.879490

[989 rows x 4 columns]


Classify each station-day into “Low”, “Medium”, or “High” demand based on trip volume percentiles.

In [54]:
# Define your query — example: Citbike Trip dataset
query = """

WITH trips_per_station_day AS (
  SELECT
    start_station_id,
    DATE(starttime) AS trip_date,
    COUNT(*) AS trip_count
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  GROUP BY start_station_id, trip_date
),

percentiles AS (
  SELECT
    PERCENTILE_CONT(trip_count, 0.33) OVER() AS p33,
    PERCENTILE_CONT(trip_count, 0.66) OVER() AS p66
  FROM trips_per_station_day
  LIMIT 1
),

demand_classification AS (
  SELECT
    t.start_station_id,
    t.trip_date,
    t.trip_count,
    CASE
      WHEN t.trip_count <= p.p33 THEN 'Low'
      WHEN t.trip_count <= p.p66 THEN 'Medium'
      ELSE 'High'
    END AS demand_category
  FROM trips_per_station_day t
  CROSS JOIN percentiles p
)

SELECT
  *
FROM
  demand_classification
ORDER BY
  start_station_id,
  trip_date;





"""

# Run the query
demand_query_job = client.query(query)

# Convert to pandas DataFrame
results = demand_query_job.to_dataframe()
print(results)

        start_station_id   trip_date  trip_count demand_category
0                   <NA>         NaT     5828994            High
1                     72  2013-07-01          55          Medium
2                     72  2013-07-02          85            High
3                     72  2013-07-03          88            High
4                     72  2013-07-04          87            High
...                  ...         ...         ...             ...
731657              3685  2018-05-27          72          Medium
731658              3686  2018-05-28          39          Medium
731659              3686  2018-05-29         185            High
731660              3686  2018-05-30         210            High
731661              3686  2018-05-31         187            High

[731662 rows x 4 columns]


Then, analyze demand category distribution by borough.

In [104]:
# 1) SQL with station_id included and no premature aggregation
query = """
-- 1) bring station_id into every trip row (fast clustered join)
WITH trips AS (
  SELECT
    t.start_station_id AS station_id,                 -- already STRING in the BQ table
    DATE(t.starttime) AS trip_date
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`   t
  JOIN `bigquery-public-data.new_york_citibike.citibike_stations` s
    ON t.start_station_name = s.name
  WHERE t.start_station_name IS NOT NULL      -- defensive
),

-- 2) daily trip counts per station
daily AS (
  SELECT
    station_id,
    trip_date,
    COUNT(*) AS trip_count
  FROM trips
  GROUP BY station_id, trip_date
),

-- 3) global tercile cut-offs
cut AS (
  SELECT
    APPROX_QUANTILES(trip_count, 100)[OFFSET(33)] AS p33,
    APPROX_QUANTILES(trip_count, 100)[OFFSET(66)] AS p66
  FROM daily
)

-- 4) tag every station-day
SELECT
  station_id,
  trip_date,
  CASE
    WHEN trip_count <= c.p33 THEN 'Low'
    WHEN trip_count <= c.p66 THEN 'Medium'
    ELSE 'High'
  END AS demand_category
FROM daily d
CROSS JOIN cut c;
"""


# 2) Pull into a DataFrame
results = client.query(query).result().to_dataframe()   # station_id is STRING

# 3) Read the TSV map
tsv_path = "citibike-stations-data-summary.tsv"   # <- edit

stations_boro = (
    pd.read_csv(tsv_path, sep="\t", dtype={'station_id': int})
      .loc[:, ['station_id', 'boro']]      # keep only what we need
      .drop_duplicates()
)


# 4) Merge & compute distribution by boro
# guarantee both columns are the same dtype
results['station_id'] = results['station_id'].astype(int)

merged = results.merge(stations_boro, on='station_id', how='left')

distribution = (
    merged
      .groupby(['boro', 'demand_category'], dropna=False)
      .size()
      .reset_index(name='station_day_count')
      .assign(
          pct_within_boro=lambda d:
              (100 * d.station_day_count /
               d.groupby('boro').station_day_count.transform('sum')).round(2)
      )
      .sort_values(['boro', 'demand_category'])
)

display(distribution.head(10))

# How many station-days still lack a borough after the merge?
missing_pct = merged['boro'].isna().mean() * 100
print(f"Missing borough label for {missing_pct:.2f}% of station-days")




/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,boro,demand_category,station_day_count,pct_within_boro
0,BK,High,8479,4.56
1,BK,Low,114327,61.48
2,BK,Medium,63154,33.96
3,MN,High,182957,50.60
4,MN,Low,57061,15.78
5,MN,Medium,121545,33.62
6,QN,High,452,2.03
7,QN,Low,18767,84.29
8,QN,Medium,3045,13.68


Missing borough label for 0.00% of station-days


On high-volume days (>90th percentile), do users ride longer or shorter trips? Does this
differ by rider type?

In [29]:
# Define your query — example: Citbike Trip dataset
query = """
-- 1) Compute daily trip counts
WITH daily_counts AS (
  SELECT
    DATE(starttime)        AS trip_date,
    COUNT(*)               AS trips_per_day
  FROM
    `bigquery-public-data.new_york_citibike.citibike_trips`
  GROUP BY
    trip_date
),

-- 2) Pull out the 90th percentile threshold
p90 AS (
  SELECT
    APPROX_QUANTILES(trips_per_day, 100)[OFFSET(90)] AS pct90
  FROM
    daily_counts
),

-- 3) Label days High-Volume vs Other
day_labels AS (
  SELECT
    dc.*,
    IF(dc.trips_per_day > p.pct90, 'High-Volume', 'Other') AS volume_label
  FROM
    daily_counts dc
  CROSS JOIN
    p90 p
),

-- 4) Join back to all trips, carry along the label
trips_with_label AS (
  SELECT
    t.tripduration,
    t.usertype,
    dl.volume_label
  FROM
    `bigquery-public-data.new_york_citibike.citibike_trips` t
  JOIN
    day_labels dl
  ON
    DATE(t.starttime) = dl.trip_date
)

-- 5) Compare avg tripduration by day‐type and usertype
SELECT
  volume_label,
  usertype,
  ROUND(AVG(tripduration)/60, 2) AS avg_duration_mins,
  COUNT(*)                 AS total_trips
FROM
  trips_with_label
GROUP BY
  volume_label,
  usertype
ORDER BY
  volume_label DESC,
  usertype;



"""

# Run the query
duration_job = client.query(query)

# Convert to pandas DataFrame
results = duration_job.to_dataframe()
print(results)

  volume_label    usertype  avg_duration_mins  total_trips
0        Other    Customer              33.85      5026543
1        Other  Subscriber              13.30     37677570
2  High-Volume    Customer              44.01      1164606
3  High-Volume  Subscriber              13.99      9240002


Additional Prompts

When are trips most frequent (by hour, day, or month)?

In [30]:
# Define your query — example: Citbike Trip dataset
hour_query = """
SELECT
  EXTRACT(HOUR FROM starttime) AS hour_of_day,
  COUNT(*)                    AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  hour_of_day
ORDER BY
  trip_count DESC
LIMIT 5;

"""

dow_query = """
SELECT
  FORMAT_TIMESTAMP('%A', starttime) AS weekday,
  COUNT(*)                          AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  weekday
ORDER BY
  trip_count DESC;

"""

month_query = """
SELECT
  EXTRACT(MONTH FROM starttime) AS month,
  COUNT(*)                     AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  month
ORDER BY
  trip_count DESC;

"""

# Run the query
hour_query_job = client.query(hour_query)
dow_query_job = client.query(dow_query)
month_query_job = client.query(month_query)


In [31]:
hour_results = hour_query_job.to_dataframe()
print(hour_results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   hour_of_day  trip_count
0         <NA>     5828994
1           17     5232747
2           18     5024723
3            8     4161676
4           16     3798414


In [32]:
dow_results = dow_query_job.to_dataframe()
print(dow_results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     weekday  trip_count
0  Wednesday     8377153
1   Thursday     8255421
2    Tuesday     8161606
3     Friday     7872756
4     Monday     7658271
5   Saturday     6540825
6     Sunday     6242689
7       None     5828994


In [33]:
month_results = month_query_job.to_dataframe()
print(month_results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


    month  trip_count
0       9     6804899
1       8     6518652
2       5     6388361
3       7     6013643
4    <NA>     5828994
5       6     5070011
6      10     4976292
7       4     4959266
8      11     3522856
9       3     2677536
10     12     2537127
11      2     1825654
12      1     1814424


In [ ]:
Top 10 most popular stations (simple aggregation)

In [34]:
# Define your query — example: Citbike Trip dataset
query = """
WITH start_ct AS (
  SELECT start_station_name AS station, COUNT(*) AS cnt
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  GROUP BY start_station_name
),
end_ct AS (
  SELECT end_station_name   AS station, COUNT(*) AS cnt
  FROM `bigquery-public-data.new_york_citibike.citibike_trips`
  GROUP BY end_station_name
)
SELECT
  station,
  SUM(cnt) AS total_trips
FROM (
  SELECT * FROM start_ct
  UNION ALL
  SELECT * FROM end_ct
)
GROUP BY
  station
ORDER BY
  total_trips DESC
LIMIT
  10;

"""

# Run the query
station_query_job = client.query(query)

# Convert to pandas DataFrame
results = station_query_job.to_dataframe()
print(results)

                 station  total_trips
0                            11657988
1     E 17 St & Broadway       867794
2  Pershing Square North       858008
3        W 21 St & 6 Ave       811777
4  West St & Chambers St       783149
5        8 Ave & W 31 St       766860
6     Broadway & E 22 St       745048
7  Lafayette St & E 8 St       744934
8     Broadway & E 14 St       688579
9        8 Ave & W 33 St       639039


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Trip frequency by hour/day/month

In [38]:
# Define your query — example: Citbike Trip dataset
f_hour_query = """
SELECT
  EXTRACT(HOUR   FROM starttime) AS hour_of_day,
  COUNT(*)                         AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  hour_of_day
ORDER BY
  hour_of_day;


"""

# Run the query
f_hour_query_job = client.query(f_hour_query)

# Convert to pandas DataFrame
results = f_hour_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


    hour_of_day  trip_count
0          <NA>     5828994
1             0      493474
2             1      274130
3             2      160739
4             3       99089
5             4       94279
6             5      297710
7             6     1121851
8             7     2352084
9             8     4161676
10            9     3346692
11           10     2339094
12           11     2470146
13           12     2871885
14           13     3012169
15           14     3103103
16           15     3257591
17           16     3798414
18           17     5232747
19           18     5024723
20           19     3517025
21           20     2353727
22           21     1647701
23           22     1242153
24           23      836519


In [41]:
# Define your query — example: Citbike Trip dataset
f_dow_query = """
SELECT
  FORMAT_TIMESTAMP('%A', starttime) AS weekday,
  COUNT(*)                          AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  weekday
ORDER BY
2 DESC

"""

# Run the query
f_dow_query_job = client.query(f_dow_query)

# Convert to pandas DataFrame
results = f_dow_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     weekday  trip_count
0  Wednesday     8377153
1   Thursday     8255421
2    Tuesday     8161606
3     Friday     7872756
4     Monday     7658271
5   Saturday     6540825
6     Sunday     6242689
7       None     5828994


In [43]:
# Define your query — example: Citbike Trip dataset
f_month_query = """
SELECT
  EXTRACT(MONTH FROM starttime) AS month,
  COUNT(*)                     AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  month
ORDER BY
  month

"""

# Run the query
f_month_query_job = client.query(f_month_query)

# Convert to pandas DataFrame
results = f_month_query_job.to_dataframe()
print(results)

    month  trip_count
0    <NA>     5828994
1       1     1814424
2       2     1825654
3       3     2677536
4       4     4959266
5       5     6388361
6       6     5070011
7       7     6013643
8       8     6518652
9       9     6804899
10     10     4976292
11     11     3522856
12     12     2537127


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


How does usage differ between members and casual riders?

In [45]:
# Define your query — example: Citbike Trip dataset
test_query = """
SELECT
  usertype,
  COUNT(*)                  AS trip_count,
  ROUND(AVG(tripduration)/60,2) AS avg_duration_mins
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  usertype;

"""

# Run the query
test_query_job = client.query(test_query)

# Convert to pandas DataFrame
results = test_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     usertype  trip_count  avg_duration_mins
0  Subscriber    46917572              13.44
1    Customer     6191149              35.76
2                 5828994                NaN


In [46]:
# Define your query — example: Citbike Trip dataset
pattern_query = """
SELECT
  usertype,
  EXTRACT(HOUR FROM starttime) AS hour,
  COUNT(*)                    AS trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  usertype, hour
ORDER BY
  usertype, hour;


"""

# Run the query
pattern_query_job = client.query(pattern_query)

# Convert to pandas DataFrame
results = pattern_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


      usertype  hour    trips
0               <NA>  5828994
1     Customer     0    61892
2     Customer     1    37299
3     Customer     2    23218
4     Customer     3    14222
5     Customer     4     9755
6     Customer     5     8600
7     Customer     6    22402
8     Customer     7    54941
9     Customer     8   126230
10    Customer     9   210230
11    Customer    10   327418
12    Customer    11   446716
13    Customer    12   518746
14    Customer    13   556523
15    Customer    14   591318
16    Customer    15   608240
17    Customer    16   594136
18    Customer    17   549561
19    Customer    18   458646
20    Customer    19   338112
21    Customer    20   232001
22    Customer    21   167923
23    Customer    22   135391
24    Customer    23    97629
25  Subscriber     0   431582
26  Subscriber     1   236831
27  Subscriber     2   137521
28  Subscriber     3    84867
29  Subscriber     4    84524
30  Subscriber     5   289110
31  Subscriber     6  1099449
32  Subscr

Are trip durations meaningfully different across user segments or times?

In [48]:
# Define your query — example: Citbike Trip dataset
avg_duration_query = """
SELECT
  usertype,
  ROUND(AVG(tripduration)/60, 2) AS avg_duration_mins,
  COUNT(*)                     AS trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  usertype;

"""

# Run the query
avg_duration_query_job = client.query(avg_duration_query)

# Convert to pandas DataFrame
results = avg_duration_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


     usertype  avg_duration_mins     trips
0  Subscriber              13.44  46917572
1    Customer              35.76   6191149
2                            NaN   5828994


In [47]:
# Define your query — example: Citbike Trip dataset
rt_hour_query = """
SELECT
  EXTRACT(HOUR FROM starttime) AS hour_of_day,
  usertype,
  ROUND(AVG(tripduration)/60, 2) AS avg_duration_mins,
  COUNT(*)                     AS trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  hour_of_day,
  usertype
ORDER BY
  hour_of_day,
  usertype;

"""

# Run the query
rt_hour_query_job = client.query(rt_hour_query)

# Convert to pandas DataFrame
results = rt_hour_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


    hour_of_day    usertype  avg_duration_mins    trips
0          <NA>                            NaN  5828994
1             0    Customer              39.69    61892
2             0  Subscriber              13.30   431582
3             1    Customer              48.73    37299
4             1  Subscriber              13.51   236831
5             2    Customer              47.91    23218
6             2  Subscriber              13.09   137521
7             3    Customer              78.73    14222
8             3  Subscriber              12.64    84867
9             4    Customer              66.20     9755
10            4  Subscriber              13.06    84524
11            5    Customer              44.99     8600
12            5  Subscriber              10.84   289110
13            6    Customer              31.63    22402
14            6  Subscriber              11.06  1099449
15            7    Customer              28.60    54941
16            7  Subscriber              12.25  

In [50]:
# Define your query — example: Citbike Trip dataset
rt_dow_query = """
SELECT
  FORMAT_TIMESTAMP('%A', starttime) AS weekday,
  usertype,
  ROUND(AVG(tripduration)/60, 2)     AS avg_duration_mins,
  COUNT(*)                         AS trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  weekday,
  usertype
ORDER BY
1

"""

# Run the query
rt_dow_query_job = client.query(rt_dow_query)

# Convert to pandas DataFrame
results = rt_dow_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


      weekday    usertype  avg_duration_mins    trips
0        None                            NaN  5828994
1      Friday    Customer              37.34   766364
2      Friday  Subscriber              13.33  7106392
3      Monday    Customer              34.75   732515
4      Monday  Subscriber              13.18  6925756
5    Saturday  Subscriber              14.42  5077299
6    Saturday    Customer              35.30  1463526
7      Sunday    Customer              34.98  1370737
8      Sunday  Subscriber              14.29  4871952
9    Thursday    Customer              36.87   638859
10   Thursday  Subscriber              13.14  7616562
11    Tuesday  Subscriber              13.09  7554246
12    Tuesday    Customer              34.66   607360
13  Wednesday  Subscriber              13.23  7765365
14  Wednesday    Customer              37.77   611788


In [51]:
# Define your query — example: Citbike Trip dataset
rt_month_query = """
SELECT
  EXTRACT(MONTH FROM starttime) AS month,
  usertype,
  ROUND(AVG(tripduration)/60, 2) AS avg_duration_mins,
  COUNT(*)                     AS trips
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  month,
  usertype
ORDER BY
  month,
  usertype;



"""

# Run the query
rt_month_query_job = client.query(rt_month_query)

# Convert to pandas DataFrame
results = rt_month_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


    month    usertype  avg_duration_mins    trips
0    <NA>                            NaN  5828994
1       1    Customer              49.91    59533
2       1  Subscriber              11.92  1754891
3       2    Customer              59.14    72165
4       2  Subscriber              12.36  1753489
5       3    Customer              43.80   188006
6       3  Subscriber              12.52  2489530
7       4    Customer              39.05   597896
8       4  Subscriber              13.57  4361370
9       5    Customer              37.49   867463
10      5  Subscriber              13.87  5520898
11      6    Customer              34.77   620060
12      6  Subscriber              13.85  4449951
13      7    Customer              31.32   937944
14      7  Subscriber              13.74  5075699
15      8    Customer              33.98  1019225
16      8  Subscriber              13.89  5499427
17      9    Customer              31.33   910797
18      9  Subscriber              13.77  5894102


Which station pairs are most frequently used?

In [171]:
# Define your query — example: Citbike Trip dataset
query = """
SELECT
  start_station_name,
  end_station_name,
  COUNT(*) AS trip_count
FROM
  `bigquery-public-data.new_york_citibike.citibike_trips`
GROUP BY
  start_station_name,
  end_station_name
ORDER BY
  trip_count DESC
LIMIT
  10;

"""

# Run the query
pair_query_job = client.query(query)

# Convert to pandas DataFrame
results = pair_query_job.to_dataframe()
print(results)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


                  start_station_name                   end_station_name  \
0                                                                         
1             Central Park S & 6 Ave             Central Park S & 6 Ave   
2  Grand Army Plaza & Central Park S  Grand Army Plaza & Central Park S   
3            Centre St & Chambers St            Centre St & Chambers St   
4                 Broadway & W 60 St                 Broadway & W 60 St   
5                   12 Ave & W 40 St              West St & Chambers St   
6                    W 21 St & 6 Ave                    9 Ave & W 22 St   
7                    W 21 St & 6 Ave                   W 22 St & 10 Ave   
8              West St & Chambers St                   12 Ave & W 40 St   
9              West St & Chambers St              West St & Chambers St   

   trip_count  
0     5828994  
1       55703  
2       25573  
3       19670  
4       19475  
5       18667  
6       17509  
7       15120  
8       14353  
9       14165 

In [196]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import glob
import os
from pathlib import Path

# 1) SQL with station_id included and no premature aggregation
bq_sql = """
WITH daily AS (
  SELECT
    t.start_station_id            AS station_id,
    t.start_station_name          AS station_name,
    DATE(t.starttime)             AS trip_date,
    t.usertype                    AS user_type,
    COUNT(*)                      AS trip_count
  FROM `bigquery-public-data.new_york_citibike.citibike_trips` t
  GROUP BY 1,2,3,4
),

cut AS (
  SELECT
    APPROX_QUANTILES(trip_count, 100)[SAFE_OFFSET(33)] AS p33,
    APPROX_QUANTILES(trip_count, 100)[SAFE_OFFSET(66)] AS p66
  FROM daily
)

SELECT
  d.*,
  CASE
    WHEN trip_count <= c.p33 THEN 'Low'
    WHEN trip_count <= c.p66 THEN 'Medium'
    ELSE 'High'
  END AS demand_category
FROM daily d
CROSS JOIN cut c
"""
# Storage API -> much faster download
bq_df = client.query(bq_sql).result().to_dataframe(bqstorage_client=True)

# ────────────────────────────────────────────────────────────────
# 2)  Local CSVs — same schema as citibike_trips
#     • folder:  ~/Downloads/missing_data
#     • gender: 0→unknown, 1→male, 2→female
# ────────────────────────────────────────────────────────────────
data_dir  = Path(tsv_path).expanduser().resolve().parent   # <‑‑ folder for both TSV + CSV
csv_files = list(data_dir.glob("*.csv"))                   # pick up every *.csv inside

csv_df_list = []
for fp in csv_files:
    df = pd.read_csv(fp, parse_dates=["Start Time", "Stop Time"], low_memory=False)


    df = (
        df.assign(
            station_id=df["Start Station ID"].astype("Int64"),
            station_name=df["Start Station Name"].astype("string"),
            trip_date=df["Start Time"].dt.date,
            gender=df["Gender"].astype("string"),
            user_type=df["User Type"].astype("string"),
            birth_year=df["Birth Year"].astype("Int64")
            
        )
        .loc[:, ["station_id", "station_name", "trip_date", "gender", "user_type", "birth_year"]]
    )

    df["gender"] = (
        df["gender"]
          .map({0: "unknown", 1: "male", 2: "female"})
          .fillna("unknown")
          .astype("string")
    )

    daily = (
        df.groupby(["station_id", "station_name", "trip_date", "gender", "user_type", "birth_year"])
          .size()
          .reset_index(name="trip_count")
    )

    csv_df_list.append(daily)

csv_df = (
    pd.concat(csv_df_list, ignore_index=True)
      .assign(src="local_csv")
)

# apply the same demand buckets used for BQ data
p33, p66 = csv_df["trip_count"].quantile([.33, .66])
csv_df["demand_category"] = pd.cut(
    csv_df["trip_count"],
    bins=[-float("inf"), p33, p66, float("inf")],
    labels=["Low", "Medium", "High"],
    right=True,
)

# ────────────────────────────────────────────────────────────────
# 3)  Stack BigQuery + CSV data, continue as before
# ────────────────────────────────────────────────────────────────
results = pd.concat([bq_df, csv_df], ignore_index=True)

results['station_id'] = (
    pd.to_numeric(results['station_id'], errors='coerce')  # force strings → numbers
      .astype('Int64')                                     # nullable int dtype
)

# ────────────────────────────────────────────────────────────────
# 2) Read TSV (station_id as STRING)
# ────────────────────────────────────────────────────────────────
stations_boro = (
    pd.read_csv(tsv_path, sep="\t", dtype={'station_id': int})
      .loc[:, ['station_id', 'boro']]
      .drop_duplicates()
)

# ────────────────────────────────────────────────────────────────
# 3) Merge
# ────────────────────────────────────────────────────────────────
stations_boro['station_id'] = stations_boro['station_id'].astype('Int64')
merged = results.merge(stations_boro, on='station_id', how='left')

# ------------------------------------------------------------------
# A)  Pivot: station × user_type  ➜ every cell = total trips
# ------------------------------------------------------------------
station_user = (
    merged
      .groupby(['station_name', 'user_type'])['trip_count']
      .sum()
      .reset_index()
)

pivot = (
    station_user
      .pivot(index='station_name', columns='user_type', values='trip_count')
      .fillna(0)
      .sort_values(by='Subscriber', ascending=False)   # or any column
)

print(pivot.head(10))          # top 10 stations by Subscriber trips

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


user_type                 Customer  Subscriber
station_name                                  
Pershing Square North  0     26347      474393
W 21 St & 6 Ave        0     18229      425091
E 17 St & Broadway     0     37743      423714
8 Ave & W 31 St        0     25585      405731
Broadway & E 22 St     0     25063      379511
Lafayette St & E 8 St  0     26169      357813
Broadway & E 14 St     0     30492      347109
8 Ave & W 33 St        0     23137      341932
W 41 St & 8 Ave        0     19628      325276
West St & Chambers St  0     84820      324990
